<a href="https://colab.research.google.com/github/camnhint/chatbot/blob/main/Nh%E1%BA%ADn_di%E1%BB%87n_t%E1%BA%A5t_c%E1%BA%A3_ch%E1%BB%AF_vi%E1%BA%BFt_tay(%C4%91ang_l%C3%A0m)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("crawford/emnist")

print("Path to dataset files:", path)

100%|██████████| 1.24G/1.24G [00:20<00:00, 64.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/crawford/emnist/versions/3


In [5]:

import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Install the idx2numpy library to read IDX files
!pip install idx2numpy

# Function to load EMNIST Letters data
def load_emnist_letters():
    # Update the path to point to the extracted folder
    data_path = os.path.join( 'emnist-letters-train-images-idx3-ubyte')
    labels_path = os.path.join( 'emnist-letters-train-labels-idx1-ubyte')

    # Load data using idx2numpy
    import idx2numpy # Import idx2numpy here, so it's available when the function is called
    data = idx2numpy.convert_from_file(data_path)
    labels = idx2numpy.convert_from_file(labels_path)

    # Data needs to be normalized and reshaped
    data = data.reshape(-1, 28, 28, 1).astype('float32') / 255.0  # 28x28, grayscale
    labels = to_categorical(labels - 1, num_classes=26)  # One-hot encode (A-Z)

    return train_test_split(data, labels, test_size=0.2, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = load_emnist_letters()

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Xây dựng mô hình CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')  # 26 lớp tương ứng A-Z
])

# Biên dịch mô hình
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

# Lưu mô hình
model.save("handwritten_character_recognition.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 99s 124ms/step - accuracy: 0.5405 - loss: 1.5422 - val_accuracy: 0.8857 - val_loss: 0.3645
Epoch 2/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 96s 123ms/step - accuracy: 0.8243 - loss: 0.5595 - val_accuracy: 0.9070 - val_loss: 0.2848
Epoch 3/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 144s 125ms/step - accuracy: 0.8597 - loss: 0.4454 - val_accuracy: 0.9193 - val_loss: 0.2468
Epoch 4/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 138s 120ms/step - accuracy: 0.8759 - loss: 0.3895 - val_accuracy: 0.9231 - val_loss: 0.2305
Epoch 5/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 144s 122ms/step - accuracy: 0.8847 - loss: 0.3596 - val_accuracy: 0.9295 - val_loss: 0.2127
Epoch 6/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 141s 121ms/step - accuracy: 0.8943 - loss: 0.3257 - val_accuracy: 0.9299 - val_loss: 0.2079
Epoch 7/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 141s 120ms/step - accuracy: 0.8980 - loss: 0.3140 - val_accuracy: 0.9313 - val_loss: 0.1994
Epoch 8/10
780/780 ━━━━━━━━━━━━━━━━━━━━ 143s 121ms/step - accuracy: 0.9046 - l

In [7]:
import cv2

# Hàm tách ký tự từ ảnh
def segment_characters(image):
    # Chuyển ảnh về grayscale
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

    # Áp dụng threshold để làm nổi bật ký tự
    _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

    # Tìm các contour của ký tự
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Lọc các contour đủ lớn để là ký tự
    characters = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 5 and h > 5:  # Lọc các vùng nhỏ
            char_img = binary[y:y + h, x:x + w]
            char_img = cv2.resize(char_img, (28, 28))  # Resize về 28x28
            characters.append(char_img)

    return characters


In [8]:
def recognize_multiple_characters(image):
    # Tách các ký tự từ ảnh
    characters = segment_characters(image)

    # Dự đoán từng ký tự
    results = []
    for char in characters:
        char = char.astype('float32') / 255.0  # Chuẩn hóa
        char = np.expand_dims(char, axis=(0, -1))  # Thêm batch size và kênh
        prediction = model.predict(char)
        predicted_char = chr(np.argmax(prediction) + ord('A'))  # Chuyển từ index về ký tự
        results.append(predicted_char)

    return ''.join(results)  # Ghép thành chuỗi kết quả


In [9]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [10]:
import gradio as gr
from PIL import Image

# Tải mô hình
from tensorflow.keras.models import load_model
model = load_model("handwritten_character_recognition.h5")

# Giao diện Gradio
def recognize_text(image):
    return recognize_multiple_characters(image)

demo = gr.Interface(
    fn=recognize_text,
    inputs=gr.Image(type="pil"),  # Đầu vào ảnh
    outputs="text",              # Kết quả đầu ra
    title="Nhận diện chữ viết tay",
    description="Tải lên ảnh chứa các ký tự viết tay (chữ hoa, chữ thường) để nhận diện."
)

if __name__ == "__main__":
    demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1ac668c44b75dd4e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
